# Motor Vehicle Collisions - Crashes
Datos sobre accidentes vehiculares en la ciudad de Nueva York
Datos hasta agosto de 2024

# Carga de datos
Una vez que tengamos nuestros datos del archivo 'Motor_Vehicle_Collisions_-_Crashes_20241009.csv', lo siguiente será localizarlos en la carpeta del proyecto, posteriormente haciendo uso de la libreria pandas, leeremos el documento y crearemos un nuevo objeto 'df' que contendra estos datos. Una vez completado podremos ver los datos.

In [13]:
import pandas as pd

In [11]:
df = pd.read_csv('./Motor_Vehicle_Collisions_-_Crashes_20241009.csv')
df

/var/folders/f2/nm0g37qj6d7bbfzhp9wv0btm0000gn/T/ipykernel_2153/1730022052.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./Motor_Vehicle_Collisions_-_Crashes_20241009.csv')


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,09/11/2021,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,12/14/2021,8:13,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2124933,08/20/2024,15:18,BROOKLYN,11212.0,NaN,NaN,NaN,NaN,NaN,44 NEW LOTS AVENUE,...,Unspecified,NaN,NaN,NaN,4750415,Sedan,Box Truck,NaN,NaN,NaN
2124934,08/21/2024,8:50,BRONX,10475.0,NaN,NaN,NaN,NaN,NaN,2035 BARTOW AVENUE,...,Unspecified,NaN,NaN,NaN,4750338,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
2124935,08/21/2024,20:10,QUEENS,11103.0,NaN,NaN,NaN,BROADWAY,NEWTOWN ROAD,NaN,...,Unspecified,NaN,NaN,NaN,4750194,Station Wagon/Sport Utility Vehicle,Moped,NaN,NaN,NaN
2124936,08/21/2024,12:58,BRONX,10462.0,NaN,NaN,NaN,WHITE PLAINS ROAD,WOOD AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4750059,Bike,Sedan,NaN,NaN,NaN
